In [1]:
import pandas as pd

In [2]:
import numpy as np

In [6]:

import pandas as pd
import numpy as np
import math
import time

def generate_dummy_requests():
    """
    Loads driver, rider, and trip data to simulate dummy rider requests,
    assigning them to the nearest available driver.
    """
    # --- 1. SETUP: Load data and initialize the world ---
    print("Setting up the simulation environment...")

    # Load the datasets from the local folder
    # Changed pd.read_csv() to pd.read_excel() for .xlsx files
    try:
        earners_df = pd.read_excel("C:/Users/91984/Downloads/earners.xlsx")
        riders_df = pd.read_excel("C:/Users/91984/Downloads/riders.xlsx")
        trips_df = pd.read_excel("C:/Users/91984/Downloads/ride_trips.xlsx")
    except FileNotFoundError as e:
        print(f"Error: A required file was not found: {e.filename}")
        print("Please ensure all three Excel files are in the Downloads folder.")
        return
    except Exception as e:
        print(f"Error loading files: {e}")
        print("Make sure the files are valid Excel files (.xlsx format).")
        return

    # Prepare the drivers from the 'earners' file
    drivers_df = earners_df[earners_df['earner_type'] == 'driver'].copy()
    drivers_df = drivers_df.set_index('earner_id')

    # Set initial status for drivers: 70% online, 30% offline
    drivers_df['status'] = np.random.choice(['online', 'offline'], size=len(drivers_df), p=[0.7, 0.3])

    # --- Set initial driver locations ---
    # We'll use the average location from real trips in city 1 as our center
    CITY_ID_TO_SIMULATE = 1
    city_trips = trips_df[trips_df['city_id'] == CITY_ID_TO_SIMULATE]

    if city_trips.empty:
        print(f"Error: No trip data found for city_id {CITY_ID_TO_SIMULATE}. Cannot set driver locations.")
        return

    city_center_lat = city_trips['pickup_lat'].mean()
    city_center_lon = city_trips['pickup_lon'].mean()

    # Scatter drivers randomly around the city center
    drivers_df['current_lat'] = city_center_lat + np.random.randn(len(drivers_df)) * 0.05
    drivers_df['current_lon'] = city_center_lon + np.random.randn(len(drivers_df)) * 0.05

    # Get a list of realistic pickup locations from the trips data
    pickup_locations = city_trips[['pickup_lat', 'pickup_lon']].dropna().drop_duplicates()

    print(f"Setup complete. Simulating for City ID: {CITY_ID_TO_SIMULATE}")
    print(f"{len(drivers_df[drivers_df['status'] == 'online'])} of {len(drivers_df)} drivers are online and ready.")
    print("-" * 50)

    # --- 2. HELPER FUNCTION: Calculate distance ---
    def haversine(lat1, lon1, lat2, lon2):
        """Calculate the distance between two points on Earth in kilometers."""
        R = 6371  # Radius of Earth
        dlat = math.radians(lat2 - lat1)
        dlon = math.radians(lon2 - lon1)
        a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c

    # --- 3. SIMULATION LOOP ---
    num_requests = 10
    for i in range(num_requests):
        print(f"\n--- Request #{i+1} ---")

        # A random rider makes a request from a random (but realistic) location
        random_rider = riders_df.sample(1).iloc[0]
        random_pickup = pickup_locations.sample(1).iloc[0]
        
        print(f"Incoming request from Rider {random_rider['rider_id']} at ({random_pickup['pickup_lat']:.4f}, {random_pickup['pickup_lon']:.4f})")

        # Find the closest available (online) driver
        online_drivers = drivers_df[drivers_df['status'] == 'online']

        if online_drivers.empty:
            print(">> No drivers are currently online. Request cannot be fulfilled.")
            time.sleep(1)
            continue

        # Calculate distance for every online driver
        distances = online_drivers.apply(
            lambda row: haversine(random_pickup['pickup_lat'], random_pickup['pickup_lon'], row['current_lat'], row['current_lon']),
            axis=1
        )

        closest_driver_id = distances.idxmin()
        min_distance = distances.min()

        # Assign the trip and update driver status
        drivers_df.loc[closest_driver_id, 'status'] = 'engaged'

        print(f">> SUCCESS: Found closest driver {closest_driver_id} ({min_distance:.2f} km away).")
        print(f">> Assigning trip. Driver {closest_driver_id} status is now 'engaged'.")
        print(f">> {len(drivers_df[drivers_df['status'] == 'online'])} drivers remain online.")
        
        time.sleep(1) # Pause for readability

    print("\n" + "-"*50)
    print("Simulation finished.")
    print("\nFinal Driver Statuses:")
    print(drivers_df['status'].value_counts())

# Run the main function when the script is executed
if __name__ == "__main__":
    generate_dummy_requests()

Setting up the simulation environment...
Setup complete. Simulating for City ID: 1
116 of 160 drivers are online and ready.
--------------------------------------------------

--- Request #1 ---
Incoming request from Rider R2544 at (52.4040, 4.9460)
>> SUCCESS: Found closest driver E10050 (1.00 km away).
>> Assigning trip. Driver E10050 status is now 'engaged'.
>> 115 drivers remain online.

--- Request #2 ---
Incoming request from Rider R2567 at (52.3674, 4.9292)
>> SUCCESS: Found closest driver E10095 (0.63 km away).
>> Assigning trip. Driver E10095 status is now 'engaged'.
>> 114 drivers remain online.

--- Request #3 ---
Incoming request from Rider R2444 at (52.3835, 4.8528)
>> SUCCESS: Found closest driver E10106 (0.79 km away).
>> Assigning trip. Driver E10106 status is now 'engaged'.
>> 113 drivers remain online.

--- Request #4 ---
Incoming request from Rider R2542 at (52.3596, 4.8987)
>> SUCCESS: Found closest driver E10152 (0.24 km away).
>> Assigning trip. Driver E10152 stat